Generating supporting files for ReLERNN

In [1]:
%run ../scripts/notebook_settings_lean.py
import glob

/faststorage/project/baboondiversity/people/eriks/second_analysis_baboons/scripts/notebook_settings_lean.py:7: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina', 'png')


In [2]:
relernn_dir = "/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/relernn_input/"

In [3]:
chrom_lengths = pd.read_csv("/home/eriks/baboondiversity/data/callability_panu3_26_04_2021/genome_lengths",
            sep="\t", names=["chrom", "length"])

In [4]:
chrom_lengths["start"] = 0
chrom_lengths = chrom_lengths[["chrom", "start", "length"]]

In [5]:
chrom_lengths.iloc[:-1]

,chrom,start,length
0,chr1,0,217458864
1,chr2,0,188283584
2,chr3,0,181338541
3,chr4,0,166074386
4,chr5,0,183742232
5,chr6,0,175400206
6,chr7,0,162899863
7,chr8,0,140346614
8,chr9,0,125784921
9,chr10,0,91330067


In [6]:
chrom_lengths.iloc[:-1].to_csv(relernn_dir + "autosome_sizes.bed", sep="\t", index=False, header=False)
chrom_lengths.iloc[-1:].to_csv(relernn_dir + "chrX_sizes.bed", sep="\t", index=False, header=False)

Creating sample ID lists to filter pops for.

In [7]:
meta_data_samples = pd.read_csv("../data/Papio_metadata_with_clustering.txt", sep =" ")
meta_data_samples_Sci = meta_data_samples.copy()
for i, row in meta_data_samples_Sci.iterrows():
    if row.PGDP_ID[0] != "P":
        meta_data_samples_Sci.at[i, "PGDP_ID"] = "Sci_"+str(row.PGDP_ID)

In [8]:
for o in meta_data_samples_Sci.Origin.unique():
    pop_subset = meta_data_samples_Sci.loc[meta_data_samples_Sci.Origin == o]
    print(o, len(pop_subset))

Mikumi, Tanzania 38
Aberdare, Kenya 2
Chunga, Zambia 27
captive 2
Awash, Ethiopia 2
Serengeti, Tanzania 14
Arusha, Tanzania 4
Ngorongoro, Tanzania 6
Ruaha, Tanzania 6
Gombe, Tanzania 17
Udzungwa, Tanzania 5
Mahale, Tanzania 7
Katavi, Tanzania 2
Selous, Tanzania 3
Lake Manyara, Tanzania 19
Tarangire, Tanzania 7
Niokolo-Koba, Senegal 12
Issa Valley, Tanzania 1
Dendro Park, Zambia 4
Filoha, Ethiopia 24
Gog Woreda, Gambella region, Ethiopia 25


In [9]:
for o in ["Dendro Park, Zambia", "Arusha, Tanzania", "Ngorongoro, Tanzania"]:
    pop_subset = meta_data_samples_Sci.loc[meta_data_samples_Sci.Origin == o]
    short_origin = o.replace(" ", ",").split(",")[0]
    pop_subset[["PGDP_ID"]].to_csv(relernn_dir + short_origin + ".sample_list.txt",
                                          header=False, index=False, sep="\t")

Checking access mask

In [10]:
mask = pd.read_csv(relernn_dir + "autosomes.sorted.bed", sep="\t", names=["chrom", "start", "end"])

In [11]:
fixed_mask = mask.loc[~((mask.start == 0) & (mask.end > 50000))]

In [16]:
fixed_mask

,chrom,start,end
0,chr1,0,73
1,chr1,74,75
2,chr1,76,77
3,chr1,80,81
4,chr1,82,89
...,...,...,...
16005255,chr20,72047499,72048329
16005256,chr20,72048358,72048359
16005257,chr20,72048403,72048774
16005258,chr20,72048781,72048782


In [18]:
for chrom in fixed_mask.chrom.unique():
    chr_subset = fixed_mask.loc[fixed_mask.chrom == chrom]
    print(sum(chr_subset.end-chr_subset.start)/chr_subset.end.max())

0.07090549778646871
0.06313863241524019
0.07328484571848408
0.06682795744311829
0.06862417454469585
0.06869392160234977
0.07012727199162838
0.06637806737539105
0.0728331577995744
0.08386837162837076
0.071320826129935
0.06295321355763549
0.06552101888070633
0.07457287294857269
0.07551994082397911
0.09234629037468911
0.07374916767598438
0.0630869337294097
0.1559295341391818
0.07627420621018373


In [19]:
fixed_mask.to_csv(relernn_dir + "autosomes.sorted.bed", sep="\t", header=False, index=False)